<a href="https://colab.research.google.com/github/RyosukeHanaoka/TechTeacher/blob/main/optPhotoFiles%E3%81%AE%E3%82%AF%E3%83%AA%E3%83%BC%E3%83%8B%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#必要なライブラリの導入

In [ ]:
from PIL import Image
import os
import shutil
!pip install pyheif

##データのバックアップ
加工した画像は別に保存するので、原本がクラッシュすることはない予定だが、二重三重に原本を保護することを心がける。このため、shutil.copytree()を用いて、OptPhotoFiles/backup/内に原本のコピーを作成する。

In [2]:
# データのバックアップ
shutil.copytree('/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/RA/RA_Right_Hand/',
                '/content/drive/MyDrive/OptPhotoFiles/backup/optPhotoFiles/RA_Right_Hand/')
shutil.copytree('/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand/',
                '/content/drive/MyDrive/OptPhotoFiles/backup/optPhotoFiles/nonRA_Right_Hand/')
shutil.copytree('/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/RA/RA_Left_Hand/',
                '/content/drive/MyDrive/OptPhotoFiles/backup/optPhotoFiles/RA_Left_Hand/')
shutil.copytree('/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand/',
                '/content/drive/MyDrive/OptPhotoFiles/backup/optPhotoFiles/nonRA_Left_Hand/')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 56.6 MB/s eta 0:00:00


'/content/drive/MyDrive/OptPhotoFiles/backup/optPhotoFiles/nonRA_Left_Hand/'

#HEICフォーマットの画像の変換
HEICフォーマットの画像が大量に含まれていたため、pyheifライブラリを用いて、これををjpegフォーマット画像に変換する関数"heic_convert_to"を定義した。

In [7]:
#heicを変換する関数を定義
def heic_convert_to(heic_filename, output_filename, q=90):
    import pyheif
    from PIL import Image

    heif = pyheif.read(heic_filename)

    image = Image.frombytes(
        heif.mode,
        heif.size,
        heif.data,
        "raw",
        heif.mode,
        heif.stride,
    )

    image.save(output_filename, quality=q)

#ファイル名の再定義関数を定義
協力者から送られたファイル名には、日本語全角文字や個人名が含まれ、その命名規則もさまざまである。このため、これ以降、個人のプロファイリングが不可能となることを受容し、ファイル名を再定義する関数、"rename_files"を定義した。

In [8]:
#ファイル名を通し番号で統一する関数"rename_files"を定義。
def rename_files(directory, prefix, start_number=1):
    counter = start_number
    for foldername, subfolders, filenames in os.walk(directory):
        for filename in filenames:
            # ファイルの絶対パスを取得
            filepath = os.path.join(foldername, filename)

            # 新しいファイル名を作成
            new_filename = f"{prefix}_{str(counter).zfill(6)}{os.path.splitext(filename)[1]}"
            new_filepath = os.path.join(foldername, new_filename)

            # ファイル名を変更
            os.rename(filepath, new_filepath)
            counter += 1

#ファイル名の再定義
"optPhotoFiles"に格納されている全ファイルを変更する。患者群では、"patients"が、健常者群では"healthy_control"が含まれるファイル番号を命名する。再定義後の画像は、OptPhotoFiles/optPhotoFiles_renamed/に格納する。

In [13]:
# 変換する画像が含まれるディレクトリパス(nonRA_右手)
os.makedirs('/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles_renamed/optPhotoFiles_nonRA_right_renamed', exist_ok=True)
input_dir = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand'
output_dir = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles_renamed/optPhotoFiles_nonRA_right_renamed'
#input_dirの内部を検索
for filename in os.listdir(input_dir):
    if filename.lower().endswith('.heic'):
        input_filepath = os.path.join(input_dir, filename)
        output_filepath = os.path.join(output_dir, os.path.splitext(filename)[0] + '.jpg')
        try:
            heic_convert_to(input_filepath, output_filepath)#関数;heic_convert_toを実行
            print(f"Converted {filename} to JPEG")#作業に成功したらこちらを表示
        except Exception as e:
            print(f"Couldn't convert {filename}: {e}")#うまくいかなければこちらを表示
    else :
        input_filepath = os.path.join(input_dir, filename)
        output_filepath = os.path.join(output_dir, os.path.splitext(filename)[0] + '.jpg')
        shutil.copy(input_filepath, output_filepath)
rename_files('/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles_renamed/optPhotoFiles_nonRA_right_renamed/', 'healthy_control')

# 変換する画像が含まれるディレクトリパス(nonRA_左手)
os.makedirs('/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles_renamed/optPhotoFiles_nonRA_left_renamed', exist_ok=True)
input_dir = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand'
output_dir = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles_renamed/optPhotoFiles_nonRA_left_renamed'
#input_dirの内部を検索
for filename in os.listdir(input_dir):
    if filename.lower().endswith('.heic'):
        input_filepath = os.path.join(input_dir, filename)
        output_filepath = os.path.join(output_dir, os.path.splitext(filename)[0] + '.jpg')

        try:
            heic_convert_to(input_filepath, output_filepath)#関数;heic_convert_toを実行
            print(f"Converted {filename} to JPEG")#作業に成功したらこちらを表示
        except Exception as e:
            print(f"Couldn't convert {filename}: {e}")#うまくいかなければこちらを表示
    else :
        input_filepath = os.path.join(input_dir, filename)
        output_filepath = os.path.join(output_dir, os.path.splitext(filename)[0] + '.jpg')
        shutil.copy(input_filepath, output_filepath)
rename_files('/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles_renamed/optPhotoFiles_nonRA_left_renamed/', 'healthy_control')

# 変換する画像が含まれるディレクトリパス(RA_右手)
os.makedirs('/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles_renamed/optPhotoFiles_RA_right_renamed', exist_ok=True)
input_dir = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/RA/RA_Right_Hand'
output_dir = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles_renamed/optPhotoFiles_RA_right_renamed'
#input_dirの内部を検索
for filename in os.listdir(input_dir):
    if filename.lower().endswith('.heic'):
        input_filepath = os.path.join(input_dir, filename)
        output_filepath = os.path.join(output_dir, os.path.splitext(filename)[0] + '.jpg')

        try:
            heic_convert_to(input_filepath, output_filepath)#関数;heic_convert_toを実行
            print(f"Converted {filename} to JPEG")#作業に成功したらこちらを表示
        except Exception as e:
            print(f"Couldn't convert {filename}: {e}")#うまくいかなければこちらを表示
    else :
        input_filepath = os.path.join(input_dir, filename)
        output_filepath = os.path.join(output_dir, os.path.splitext(filename)[0] + '.jpg')
        shutil.copy(input_filepath, output_filepath)
rename_files('/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles_renamed/optPhotoFiles_RA_right_renamed/', 'patients')

# 変換する画像が含まれるディレクトリパス(RA_左手)
os.makedirs('/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles_renamed/optPhotoFiles_RA_left_renamed', exist_ok=True)
input_dir = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/RA/RA_Left_Hand'
output_dir = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles_renamed/optPhotoFiles_RA_left_renamed'
#input_dirの内部を検索
for filename in os.listdir(input_dir):
    if filename.lower().endswith('.heic'):
        input_filepath = os.path.join(input_dir, filename)
        output_filepath = os.path.join(output_dir, os.path.splitext(filename)[0] + '.jpg')

        try:
            heic_convert_to(input_filepath, output_filepath)#関数;heic_convert_toを実行
            print(f"Converted {filename} to JPEG")#作業に成功したらこちらを表示
        except Exception as e:
            print(f"Couldn't convert {filename}: {e}")#うまくいかなければこちらを表示
    else :
        input_filepath = os.path.join(input_dir, filename)
        output_filepath = os.path.join(output_dir, os.path.splitext(filename)[0] + '.jpg')
        shutil.copy(input_filepath, output_filepath)
rename_files('/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles_renamed/optPhotoFiles_RA_left_renamed/', 'patients')

Converted IMG_1797 - Ho2O hO2O.HEIC to JPEG
Converted IMG_2324 - Yuuki Uchida.HEIC to JPEG
Couldn't convert IMG_6586 - あこすあこす.HEIC: Input is not a HEIF/AVIF file
Converted IMG_1907 - Manami Okamura.HEIC to JPEG
Converted IMG_9517 - kujilife.HEIC to JPEG
Converted IMG_7199 (2) - Akira Ishida.HEIC to JPEG
Converted IMG_9772 - 虎金六知ひーやん.HEIC to JPEG
Converted 85597DAA-3D79-48F4-9D3A-B2D814E104C2_1_201_a - u akitakeru.heic to JPEG
Converted IMG_3830 - TA KE.HEIC to JPEG
Converted CFB65066-75A1-41BA-8A5F-647920ED0899 - T T (瀧).heic to JPEG
Converted IMG_8542 - 長尾早希.HEIC to JPEG
Converted IMG_5739 - 黒川京子.HEIC to JPEG
Converted IMG_1981 - Ayaka Saito.HEIC to JPEG
Converted IMG_3200 - 星空.HEIC to JPEG
Converted 4F665B36-1D03-40F8-BCD6-6FE616E12B22 - 茅原陽子.heic to JPEG
Converted IMG_7850 - 稲美.HEIC to JPEG
Converted IMG_7729 - mayu.HEIC to JPEG
Converted IMG_3821 - KT_DB.HEIC to JPEG
Converted IMG_0350 - Yuki Nakazawa.HEIC to JPEG
Converted 9ACDA11C-8900-4391-B5CA-CB03144BE301 - 桃式会社Liam.heic to JP

#背景ノイズ消去
撮影された画像には、さまざまなレベルで背景に色々なものが写り込んでいる。このままでは背景画像の相違によって特徴量が抽出されてしまう危険性を否定できない。このため、セマンティックセグメンテーションの手法を使って、背景画像を白一色に統一する。
デフォルトではMediaPipeに用意されているdeeplab_v3を搭載したモデルを使用しているが、今後はdeeplab_v3を独自に訓練したモデルを使用できるようにする。

In [ ]:
!pip install -q mediapipe==0.10.0
!wget -O deeplabv3.tflite -q https://storage.googleapis.com/mediapipe-models/image_segmenter/deeplab_v3/float32/1/deeplab_v3.tflite

In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# 入力および出力ディレクトリを設定
base_dir = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/'
output_base_dir = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/segmented_photo/'
sub_dirs = ['RA/RA_Right_Hand', 'RA/RA_Left_Hand', 'nonRA/nonRA_Right_Hand', 'nonRA/nonRA_Left_Hand']
output_sub_dirs = ['RA/RA_Right_Hand', 'RA/RA_Left_Hand', 'nonRA/nonRA_Right_Hand', 'nonRA/nonRA_Left_Hand']

# 画像のパスを取得
image_paths = []
for sub_dir in sub_dirs:
    full_dir = os.path.join(base_dir, sub_dir)
    for filename in os.listdir(full_dir):
        if filename.endswith('.jpg'):
            image_path = os.path.join(full_dir, filename)
            image_paths.append(image_path)

# 出力ディレクトリを作成
for sub_dir in output_sub_dirs:
    os.makedirs(os.path.join(output_base_dir, sub_dir), exist_ok=True)

# 入力と出力のディレクトリ
#base_dir = '/path/to/input'
#output_base_dir = '/path/to/output'
#os.makedirs(output_base_dir, exist_ok=True)

# セグメンテーションオプションの設定
base_options = python.BaseOptions(model_asset_path='deeplabv3.tflite')
options = vision.ImageSegmenterOptions(base_options=base_options,
                                       output_category_mask=True)

# セグメンテーションモデルのロード
with vision.ImageSegmenter.create_from_options(options) as segmenter:
    # 入力ディレクトリ内のすべての画像ファイルに対して
    for filename in os.listdir(base_dir):
        if filename.endswith('.jpg'):
            image_path = os.path.join(base_dir, filename)
            image = cv2.imread(image_path)

            # MediaPipeのImageオブジェクトを作成
            mp_image = mp.Image(image=image, color_format=mp.ColorFormat.BGR)

            # セグメンテーションの実行
            segmentation_result = segmenter.segment(mp_image)
            category_mask = segmentation_result.category_mask.numpy_view()

            # 背景を白に、手の部分は元の画像を使用
            output_image = np.where(
                np.expand_dims(category_mask, axis=-1),
                image,  # Hand part, keep original pixels
                [255, 255, 255]  # Background part, set to white
            )

            # 出力画像の保存
            output_path = os.path.join(output_base_dir, filename)
            cv2.imwrite(output_path, output_image)